In [33]:
!pip install datasets transformers evaluate

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [37]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [38]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [40]:
#Pasta do modelo pré-treinado
model_path = "google-bert/bert-base-uncased"

#Carregando o modelo Tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Carregar o modelo com a classificação binária das labels
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}

model = AutoModelForSequenceClassification.from_pretrained(model_path,
num_labels=2,
id2label=id2label,
label2id=label2id,)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# Freeze todo os parametros do modelo base

for name, param in model.base_model.named_parameters():
	param.requires_grad = False

# Unfreeze da camada de polingo do modelo base
for name, param in model.base_model.named_parameters():
	if "pooler" in name:
		param.requires_grad = True

In [42]:
def preprocess_function(examples):
	return tokenizer(examples["text"], truncation=True)

tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [43]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [44]:
accuracy = evaluate.load("accuracy")

auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
	#Pega as predições
	predictions, labels = eval_pred

	#Aplica o soft max para as predições pegas
	probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)

	#Usa probabilidade para classe positiva para ROC AUC
	positive_class_probs = probabilities[:, 1]

	# Computa AUC

	auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

	#Muitas predições classes probabilitistacas

	predicted_classes = np.argmax(predictions, axis=1)

	acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'], 3)

	return {"Accuracy": acc, "AUC": auc}


In [46]:
#Hyper parametros

lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
	output_dir="bert-phishing-classifier_teacher",
	learning_rate=lr,
	per_device_train_batch_size=batch_size,
	per_device_eval_batch_size=batch_size,
	num_train_epochs=num_epochs,
	logging_strategy="epoch",
	eval_strategy="epoch",
	save_strategy="epoch",
	load_best_model_at_end=True,

)

In [47]:
trainer = Trainer(
	model=model,
	train_dataset=tokenized_data["train"],
	eval_dataset=tokenized_data["test"],
	tokenizer=tokenizer,
	data_collator=data_collator,
	compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-47-e85bcb109bd1>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,0.536300


TrainOutput(global_step=789, training_loss=0.5092842479169142, metrics={'train_runtime': 1540.1338, 'train_samples_per_second': 4.091, 'train_steps_per_second': 0.512, 'total_flos': 210747844233120.0, 'train_loss': 0.5092842479169142, 'epoch': 3.0})

In [48]:
predictions = trainer.predict(tokenized_data["validation"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': 0.8, 'AUC': 0.9}
